In [1]:
import pandas as pd

In [4]:
df = pd.read_csv(r'/Users/admin/bank+marketing/bank/bank-full.csv', sep = ';')

df.head(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [11]:
columns = [
'age',
'job',
'marital',
'education',
'balance',
'housing',
'contact',
'day',
'month',
'duration',
'campaign',
'pdays',
'previous',
'poutcome',
'y'  
]

df = df[columns]

In [26]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [12]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

### Q1. What is the most frequent observation (mode) for the column education?

In [13]:
df.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

In [23]:
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

### Q2. Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features. What are the two features that have the biggest correlation?

In [31]:
corr_matrix = df[numerical].corr()

corr_matrix_unstacked = corr_matrix.unstack().sort_values(ascending=False)

highest_corr = corr_matrix_unstacked[corr_matrix_unstacked < 1].idxmax()

highest_corr

('previous', 'pdays')

In [41]:
corr_df = pd.DataFrame(corr_matrix_unstacked, columns=['Correlation'])
corr_df.reset_index(inplace=True)
corr_df.columns = ['Feature_1', 'Feature_2', 'Correlation']

corr_df[corr_df.Correlation < 1]

,Feature_1,Feature_2,Correlation
7,previous,pdays,0.454820
8,pdays,previous,0.454820
9,campaign,day,0.162490
10,day,campaign,0.162490
11,age,balance,0.097783
12,balance,age,0.097783
13,balance,duration,0.021560
14,duration,balance,0.021560
15,balance,previous,0.016674
16,previous,balance,0.016674


In [42]:
# df.y = (df.y == 'yes').astype(int)

In [43]:
from sklearn.model_selection import train_test_split

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

In [44]:
y_test = df_test.y.values
y_train = df_train.y.values
y_val = df_val.y.values

In [45]:
del df_train['y']
del df_val['y']
del df_test['y']

In [47]:
df_train.columns

Index(['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact',
       'day', 'month', 'duration', 'campaign', 'pdays', 'previous',
       'poutcome'],
      dtype='object')

### Question 3 Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only. Round the scores to 2 decimals using round(score, 2).

In [9]:
len(df_train), len(df_test), len(df_val)

(27126, 9043, 9042)

In [122]:
9043 / (27126 + 9043 + 9042)

0.20001769480878548

In [50]:
df_train.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
dtype: object

In [52]:
categorical = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [18]:
from sklearn.metrics import mutual_info_score

In [57]:
def calculate_mi(series):
    return mutual_info_score(series, y_train)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi)


,MI
poutcome,0.029533
month,0.025090
contact,0.013356
housing,0.010343
job,0.007316
education,0.002697
marital,0.002050


In [56]:
# Import necessary libraries
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder

# Assume df_train is the training set DataFrame

# Selecting categorical columns (excluding 'y' which is the target variable)
categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

# Encoding categorical variables to integers
df_encoded = df_train[categorical_columns].apply(LabelEncoder().fit_transform)

# Encoding target variable 'y'
y_encoded = LabelEncoder().fit_transform(y_train)

# Calculating mutual information between y and the categorical variables
mi_scores = mutual_info_classif(df_encoded, y_encoded)

# Rounding the scores to 2 decimal places
mi_scores_rounded = [round(score, 2) for score in mi_scores]

# Creating a DataFrame for the mutual information scores
mi_df = pd.DataFrame({
    'Categorical_Variable': categorical_columns,
    'Mutual_Information_Score': mi_scores_rounded
})

# Display the DataFrame with scores
print(mi_df)


  Categorical_Variable  Mutual_Information_Score
0                  job                      0.01
1              marital                      0.00
2            education                      0.01
3              housing                      0.02
4              contact                      0.02
5                month                      0.03
6             poutcome                      0.03


### Q4 Now let's train a logistic regression. Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [58]:
from sklearn.feature_extraction import DictVectorizer

In [63]:
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [64]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

DictVectorizer(sparse=False)

In [65]:
X_train = dv.transform(train_dict)

In [66]:
from sklearn.linear_model import LogisticRegression

In [69]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [70]:
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [71]:
model.predict_proba(X_val)

array([[0.98705881, 0.01294119],
       [0.99027041, 0.00972959],
       [0.84655921, 0.15344079],
       ...,
       [0.94755389, 0.05244611],
       [0.99090938, 0.00909062],
       [0.71834612, 0.28165388]])

In [82]:
y_pred = model.predict_proba(X_val)[:, 1]

In [83]:
y_pred

array([0.01294119, 0.00972959, 0.15344079, ..., 0.05244611, 0.00909062,
       0.28165388])

In [84]:
subscr = y_pred > 0.5

In [85]:
(y_val == subscr).mean()

0.9007962840079629

In [101]:
dict(zip(dv.get_feature_names_out(), model.coef_[0].round(5)))

{'age': -0.00373,
 'balance': 1e-05,
 'campaign': 0.00705,
 'contact=cellular': 0.00419,
 'contact=telephone': -0.08521,
 'contact=unknown': 0.0001,
 'day': 0.00856,
 'duration': -0.38147,
 'education=primary': -0.35682,
 'education=secondary': -0.45854,
 'education=tertiary': -0.17065,
 'education=unknown': 0.25158,
 'housing=no': -0.23133,
 'housing=yes': -0.22306,
 'job=admin.': 0.29497,
 'job=blue-collar': -0.22216,
 'job=entrepreneur': -0.17449,
 'job=housemaid': -0.15241,
 'job=management': -0.28479,
 'job=retired': -0.05034,
 'job=self-employed': 0.04628,
 'job=services': 0.20508,
 'job=student': 0.0383,
 'job=technician': -0.75965,
 'job=unemployed': -0.10589,
 'job=unknown': -1.67434,
 'marital=divorced': -0.84917,
 'marital=married': 0.38962,
 'marital=single': -0.45806,
 'month=apr': -1.37888,
 'month=aug': -1.09333,
 'month=dec': 0.24614,
 'month=feb': 1.41168,
 'month=jan': -0.61426,
 'month=jul': -1.06891,
 'month=jun': 0.74402,
 'month=mar': 0.72795,
 'month=may': 0.3083

In [89]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
import pandas as pd

# Assume df is your dataset

# Splitting the dataset into training and validation sets (80% training, 20% validation)
X = df.drop(columns=['y'])  # Features
y = df['y']                 # Target variable

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# One-hot encode the categorical variables
categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

# Initialize the OneHotEncoder
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Updated to sparse_output

# Fit and transform on the training set
X_train_encoded = encoder.fit_transform(X_train[categorical_columns])

# Transform the validation set
X_val_encoded = encoder.transform(X_val[categorical_columns])

# Drop the original categorical columns from the dataset and concatenate with the one-hot encoded columns
X_train_numeric = X_train.drop(columns=categorical_columns).values
X_val_numeric = X_val.drop(columns=categorical_columns).values

# Concatenate the numeric and encoded categorical data
import numpy as np
X_train_final = np.hstack([X_train_numeric, X_train_encoded])
X_val_final = np.hstack([X_val_numeric, X_val_encoded])

# Train the logistic regression model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_final, y_train)

# Make predictions on the validation set
y_pred = model.predict(X_val_final)

# Calculate the accuracy
accuracy = accuracy_score(y_val, y_pred)

# Round the accuracy to 2 decimal places
accuracy_rounded = round(accuracy, 2)

# Output the result
print(f"Validation accuracy: {accuracy_rounded}")


Validation accuracy: 0.9


In [93]:
# Train the model on all features (same as Q4)
model.fit(X_train_final, y_train)

# Get the accuracy on the validation set with all features
accuracy_all_features = accuracy_score(y_val, model.predict(X_val_final))

# Create a list to store the accuracy differences
accuracy_diff_list = []

# Loop through each feature and train a model without it
for i in range(X_train_final.shape[1]):
    # Remove the i-th column from the training and validation sets
    X_train_temp = np.delete(X_train_final, i, axis=1)
    X_val_temp = np.delete(X_val_final, i, axis=1)

    # Train the model without the i-th feature
    model.fit(X_train_temp, y_train)

    # Calculate the accuracy without the i-th feature
    accuracy_without_feature = accuracy_score(y_val, model.predict(X_val_temp))

    # Calculate the difference in accuracy
    accuracy_difference = accuracy_all_features - accuracy_without_feature

    # Store the result
    accuracy_diff_list.append({'Feature': feature_names[i], 'Accuracy_Difference': accuracy_difference})

# Convert the list to a DataFrame
accuracy_diff = pd.DataFrame(accuracy_diff_list)

# Find the feature with the smallest difference
least_useful_feature = accuracy_diff.loc[accuracy_diff['Accuracy_Difference'].idxmin()]

# Output the least useful feature
print(f"The least useful feature is: {least_useful_feature['Feature']} with an accuracy difference of {least_useful_feature['Accuracy_Difference']:.4f}")


The least useful feature is: job_student with an accuracy difference of -0.0009


In [98]:
accuracy_diff[accuracy_diff.Feature.isin(['age','balance', 'marital', 'previous'])]

,Feature,Accuracy_Difference
0,age,0.000553
1,balance,-0.000553
6,previous,-0.000111


In [99]:
accuracy_diff 

,Feature,Accuracy_Difference
0,age,0.000553
1,balance,-0.000553
2,day,0.000663
3,duration,0.009068
4,campaign,0.001216
5,pdays,0.000221
6,previous,-0.000111
7,job_blue-collar,0.000221
8,job_entrepreneur,-0.000221
9,job_housemaid,0.000332


In [111]:
from sklearn.model_selection import train_test_split

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

y_test = df_test.y.values
y_train = df_train.y.values
y_val = df_val.y.values

del df_train['y']
del df_val['y']
del df_test['y']

categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

for i in (categorical_columns + numerical):
    columns_list = categorical_columns + numerical
    columns_list.remove(i)
    # print(i, len(columns_list))
    df_train_w = df_train[columns_list]
    train_dict_w = df_train_w.to_dict(orient='records')
    dv_w = DictVectorizer(sparse=False)
    dv_w.fit(train_dict_w)

    X_train_w = dv_w.transform(train_dict)

    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_w, y_train)

    val_dict = df_val[columns_list].to_dict(orient='records')
    X_val = dv_w.transform(val_dict)

    model.predict_proba(X_val)

    y_pred = model.predict_proba(X_val)[:, 1]

    subscr = y_pred > 0.5

    if i in ['age', 'balance', 'marital', 'previous']:
        print(i, 0.9007962840079629 - (y_val == subscr).mean())
    




marital 0.00022119000221187957
age -0.00022119000221187957
balance -0.00022119000221187957
previous -0.0005529750055297544


### Q6 Now let's train a regularized logistic regression. Which of these C leads to the best accuracy on the validation set?
    

In [115]:
from sklearn.model_selection import train_test_split

df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=42)

y_test = df_test.y.values
y_train = df_train.y.values
y_val = df_val.y.values

del df_train['y']
del df_val['y']
del df_test['y']

categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']
numerical = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

Cs = [0.01, 0.1, 1, 10, 100]



for s in Cs:
    # print(i, len(columns_list))
    df_train_w = df_train[columns_list]
    train_dict_w = df_train_w.to_dict(orient='records')
    dv_w = DictVectorizer(sparse=False)
    dv_w.fit(train_dict_w)

    X_train_w = dv_w.transform(train_dict)

    model = LogisticRegression(solver='liblinear', C=s, max_iter=1000, random_state=42)
    model.fit(X_train_w, y_train)

    val_dict = df_val[columns_list].to_dict(orient='records')
    X_val = dv_w.transform(val_dict)

    model.predict_proba(X_val)

    y_pred = model.predict_proba(X_val)[:, 1]

    subscr = y_pred > 0.5
    
    print(s, round((y_val == subscr).mean(), 3))
    




0.01 0.898
0.1 0.901
1 0.901
10 0.901
100 0.901


In [119]:
df = pd.read_csv(r'/Users/admin/bank+marketing/bank/bank-full.csv', sep = ';')

# Encode the target variable 'y' with 1 for 'yes' and 0 for 'no'
df['y'] = df['y'].map({'yes': 1, 'no': 0})


from sklearn.model_selection import train_test_split

# Define the columns to use
columns_to_use = ['age', 'job', 'marital', 'education', 'balance', 'housing', 
                  'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 
                  'previous', 'poutcome']

# Split the data into features (X) and target (y)
X = df[columns_to_use]
y = df['y']

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Define categorical columns
categorical_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

# One-hot encode the categorical features
encoder = ColumnTransformer(transformers=[('cat', OneHotEncoder(), categorical_columns)], remainder='passthrough')

# Transform the datasets
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_val)
X_test_encoded = encoder.transform(X_test)


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Initialize the LogisticRegression model
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

# Train the model on the training data
model.fit(X_train_encoded, y_train)

# Predict on the validation set
y_pred_val = model.predict(X_val_encoded)

# Calculate accuracy on the validation set
accuracy_val = accuracy_score(y_val, y_pred_val)

# Output the accuracy rounded to 3 decimal places
print(f"Validation accuracy: {accuracy_val:.3f}")


# List of C values to try
C_values = [0.01, 0.1, 1, 10, 100]

# Dictionary to store accuracies for each C
accuracy_dict = {}

# Loop through each C value
for C in C_values:
    # Initialize the LogisticRegression model with current C
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    
    # Train the model
    model.fit(X_train_encoded, y_train)
    
    # Predict on the validation set
    y_pred_val = model.predict(X_val_encoded)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_val, y_pred_val)
    
    # Store the accuracy for this C value
    accuracy_dict[C] = accuracy

    # Print the accuracy for this value of C
    print(f"C={C}: Accuracy={accuracy:.3f}")

# Find the C with the best accuracy
best_C = max(accuracy_dict, key=accuracy_dict.get)
best_accuracy = accuracy_dict[best_C]

print(f"\nThe best value of C is: {best_C} with accuracy: {best_accuracy:.3f}")



Validation accuracy: 0.901
C=0.01: Accuracy=0.899
C=0.1: Accuracy=0.901
C=1: Accuracy=0.901
C=10: Accuracy=0.901
C=100: Accuracy=0.901

The best value of C is: 0.1 with accuracy: 0.901


In [118]:
# Check for missing values in y
missing_y = df['y'].isna().sum()
print(f"Number of missing values in 'y': {missing_y}")


Number of missing values in 'y': 45211
